In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os


In [2]:
#import the mobilenetv2 without the last layer
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),weights="imagenet",include_top=False)

In [3]:
#add other layers
x = tf.keras.layers.GlobalAvgPool2D()(base_model.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
#the output layer for binary classification
x = tf.keras.layers.Dense(1,activation="sigmoid")(x)

model = tf.keras.Model(inputs = base_model.input , outputs = x)


In [4]:
#freezing the original model's layers
for layer in base_model.layers:
    layer.trainable = False

In [5]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [6]:
# define the directory containing training images
directory = 'Train'
target_size = (224, 224)

# initialize lists to store images and labels
images = []
labels = []

# iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join(directory, filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
        # add the preprocessed image to the list of images
        images.append(img_array)
        
        # assuming filenames follow a specific convention (e.g., dog_001.jpg, cat_002.jpg)
        if filename.startswith('dog'):
            labels.append(1)  # 1 for dog
        else:
            labels.append(0)  # 0 for cat

# convert lists to numpy arrays
X_train = np.array(images)
Y_train = np.array(labels)

In [7]:
#train the final model
model.fit(X_train,Y_train,epochs=10)

Epoch 1/10
2/2 [==============================] - 5s 519ms/step - loss: 0.7317 - accuracy: 0.5312
Epoch 2/10
2/2 [==============================] - 1s 608ms/step - loss: 0.3479 - accuracy: 0.9375
Epoch 3/10
2/2 [==============================] - 1s 570ms/step - loss: 0.1528 - accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 1s 554ms/step - loss: 0.0626 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 1s 754ms/step - loss: 0.0224 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 1s 729ms/step - loss: 0.0174 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 1s 850ms/step - loss: 0.0082 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 1s 673ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 1s 760ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 1s 679ms/step - loss: 0.0012 - accuracy: 1.0000


In [8]:
# define the directory containing test images
directory = 'Test'
target_size = (224, 224)

# initialize lists to store images and labels
images = []
labels = []

# iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join(directory, filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
        # add the preprocessed image to the list of images
        images.append(img_array)
        
        # assuming filenames follow a specific convention (e.g., dog_001.jpg, cat_002.jpg)
        if filename.startswith('dog'):
            labels.append(1)  # 1 for dog
        else:
            labels.append(0)  # 0 for cat

# convert lists to numpy arrays
X_test = np.array(images)
Y_test = np.array(labels)

In [9]:
#make predictions for the test set
predictions = model.predict(X_test)
predictions = np.reshape(predictions, (19,))
predictions = (predictions > 0.5).astype(int)

1/1 [==============================] - 2s 2s/step


In [10]:
#calculate the accuracy of the model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test, predictions)

In [11]:
print(accuracy)

1.0
